# Creating Test Set

In this notebook we generate some artificial example image to check the model and the created pipeline functionality.  
Our model is trained to detect holes and scratches on a workpiece of the shopfloor, so the test set will contain images with circles at certain positions (representing the intended holes) and/or lines (representing unintended scracthes on the board).

### Create Dataset

First we create the 'valid board' which only contains the 7 intended holes and no scratches, with a background image of size (224x224).

In [ ]:
from PIL import Image

_canvas_size = (224, 224)
_WHITE = (255, 255, 255)
_BLACK = (0, 0, 0)
_BLUE = (68,115,197)

board = Image.open("../data/raw/board.png")
print("board size:", board.size)
canvas = Image.new(mode="RGB", size=_canvas_size, color=_WHITE)
print("canvas size:", canvas.size)
board.show(), canvas.show()

Now let's create a method to rotate the board on the background with random angle!

In [ ]:
from random import randint

def randomize(original: Image):
    canvas = Image.new(mode="RGB", size=_canvas_size, color=_WHITE)
    delta_x = canvas.size[0] - original.size[0]
    rnd_x = randint(10, delta_x - 10)
    delta_y = canvas.size[1] - original.size[1]
    rnd_y = randint(10, delta_y  - 10)
    rotation = randint(-10,10)

    canvas.paste(original.rotate(rotation, expand=True, fillcolor=_WHITE), (rnd_x, rnd_y))
    return canvas

valid_board = randomize(board)
valid_board.show()

In [ ]:
from pathlib import Path

labels = ["VALID", "EXTRA_HOLE", "MISSING_HOLE", "DAMAGED"]  
images_path = Path("../data/processed/")
training_set_size = 10

Now we can create some image variation for valid boards and will save into "VALID" folder.

In [ ]:

valid_board = Path(images_path / labels[0])
valid_board.mkdir(parents=True, exist_ok=True)

for i in range(training_set_size):
    image = randomize(board)
    image.save(valid_board / f"valid_{i}.jpg")

Let's define a method to create extra hole on the board!

In [ ]:
from PIL import ImageDraw

def draw_extra_hole(original: Image):
    original = original.convert("RGBA")
    draw = ImageDraw.Draw(original)
    r = 10
    x = randint(20, original.size[0] - 20)
    y = randint(20, original.size[1] - 20)
    lu_point = (x - r, y - r)
    rb_point = (x + r, y + r)
    draw.ellipse([lu_point, rb_point], fill=_BLACK)

    return original.convert("RGB")

extra_hole = draw_extra_hole(board)
extra_hole.show()

In [ ]:
# Extra holes on board
extra_hole = Path(images_path / labels[1])
extra_hole.mkdir(parents=True, exist_ok=True)

for i in range(training_set_size):
    image = draw_extra_hole(board)
    rotated = randomize(image)
    rotated.save(extra_hole / f"extra_hole_{i}.jpg")

Let's define a method to delete a hole from the board!

In [ ]:
from PIL import ImageDraw

def delete_hole(original: Image):
    holes = [
        [(13,16),(33,36)],
        [(46,18),(66,38)],
        [(78,17),(98,37)],
        [(110,17),(130,17)],

        [(13,51),(33,71)],
        [(14,83),(34,103)],
        [(13,111),(33,131)],
        [(110,150),(130,170)],
    ]
    original = original.convert("RGBA")
    draw = ImageDraw.Draw(original)
    r = randint(0, len(holes) - 1)
    draw.ellipse(holes[r], fill=_BLUE)

    return original.convert("RGB")

missing_hole = delete_hole(board)
missing_hole.show()

In [ ]:
# Missing holes on board
missing_hole = Path(images_path / labels[2])
missing_hole.mkdir(parents=True, exist_ok=True)

for i in range(training_set_size):
    image = delete_hole(board)
    rotated = randomize(image)
    rotated.save(missing_hole / f"extra_hole_{i}.jpg")

Let's define a method to create scratches on the board!  

In [ ]:
from matplotlib import pyplot as plt

def draw_lines(original: Image):
    original = original.convert("RGBA")
    draw = ImageDraw.Draw(original)
    
    points = []
    for i in range(randint(2,6)):
        x1 = randint(36, original.size[0] - 36)
        y1 = randint(36, original.size[1] - 36)
        _point = (x1, y1)
        points.append(_point)

    draw.line(points, fill=_BLACK, width=2)

    return original.convert("RGB")

lined = draw_lines(board)
plt.imshow(lined)

In [ ]:
# Extra lines on board
extra_line = Path(images_path / labels[3])
extra_line.mkdir(parents=True, exist_ok=True)

for i in range(training_set_size):
    image = draw_lines(board)
    rotated = randomize(image)
    rotated.save(extra_line / f"extra_hole_{i}.jpg")

# Test set is ready now

Now we have created a set of images for all of interesting cases, and they will be used in the following notebooks.  